In [2]:
import pandas as pd
from konlpy.tag import Twitter

In [3]:
naver_user = pd.read_csv('[Final]naver_user_table.csv',sep='\t',encoding='utf-8')
dbdbdeep_user = pd.read_csv('[Final]dbdbdeep_user_table.csv',sep='\t',encoding='utf-8')

In [4]:
def refined_category(category):
    refined_list = []
    for i in category.split('], '):
        try:
            category_count = int(i.split(', ')[0].replace('[','').replace("'",''))
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((category_count,categoty_title))
        except:
            pass
    return refined_list

In [5]:
def refined_category_title(category):
    refined_list = []
    for i in category.split('], '):
        try:
            categoty_title = i.split(', ')[1]
            categoty_title = i.split(', ')[1].replace(']]','')
            categoty_title = categoty_title.replace(')]','')
            refined_list.append((categoty_title))
        except:
            pass
    return refined_list

In [6]:
korean_stopword = pd.read_csv('Korean_stopwords.txt',encoding='utf-8',header=None)
korean_stopword_list = korean_stopword[0].tolist()
stopwords = ['하다', ',', '들', '이', '..', '.', '것', '다', '이다', '~', '그', '그녀', '저', '...', '"', '~~',"'",':','&','[',']','{','}','(',')','-'] 
sum_stopwords = stopwords + korean_stopword_list

In [7]:
def Korean_tagger(Text):

    twitter = Twitter()

    try:
        
        
        malist = twitter.pos(Text, norm=True, stem=True)
        r = []
        remove_tag =['Eomi','PreEomi','Josa','Determiner','Foreign','Alpha','Number','Punctuation','Suffix','Unknown','Hashtag','KoreanParticle','ScreenName']

        stopwords = sum_stopwords


        for word in malist:
            # 어미/조사/구두점/ㅋㅋ^^ㅎㅎ/음표살림/Alphabet/부사는 대상에서 제외
            if not word[1] in remove_tag:
                if not word[0] in r:
                    if not word[0] in stopwords:
                       # 숫자, 특수문자 제거.
                        r.append(word[0])
        return ' '.join(r)
    except Exception as e:
        return ''

In [21]:
naver_user['Categories_embedding'] = naver_user['Categories'].apply(lambda x : "".join(refined_category_title(x)))
naver_user['Categories_embedding'] = naver_user['Categories_embedding'].apply(lambda x : Korean_tagger(x))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [23]:
dbdbdeep_user['Categories_embedding'] = dbdbdeep_user['Categories'].apply(lambda x : "".join(refined_category_title(x)))
dbdbdeep_user['Categories_embedding'] = dbdbdeep_user['Categories_embedding'].apply(lambda x : Korean_tagger(x))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [24]:
Total_user = pd.concat([naver_user,dbdbdeep_user])

In [25]:
Total_test = Total_user[['Categories_embedding','Credibility']]

In [27]:
Categories_text = Total_test['Categories_embedding'].tolist()

## Tfidf

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=391)
x = v.fit_transform(Categories_text)

In [33]:
y = Total_test['Credibility'].tolist()

In [34]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [35]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.830039525692


In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

          0       0.85      0.91      0.88       672
          1       0.79      0.67      0.73       340

avg / total       0.83      0.83      0.83      1012



In [37]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(result, x, y, cv=5)
scores                                              

array([ 0.85504886,  0.84201954,  0.72430669,  0.8120915 ,  0.81372549])

## Doc2vec

In [38]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [39]:
Total_test = Total_user[['Categories_embedding','Credibility']]

In [40]:
text = Total_test['Categories_embedding'].tolist()

In [41]:
sentences = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text)]

In [42]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [43]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [44]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=391,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,   # negative sampling / default 5
)

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-09-02 11:46:54,549 : WARNING : consider setting layer size to a multiple of 4 for greater performance


In [45]:
doc_vectorizer.build_vocab(sentences)

2018-09-02 11:46:56,973 : INFO : collecting all words and their counts
2018-09-02 11:46:56,975 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-09-02 11:46:56,977 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-09-02 11:46:57,063 : INFO : collected 1337 word types and 3065 unique tags from a corpus of 3065 examples and 229662 words
2018-09-02 11:46:57,063 : INFO : Loading a fresh vocabulary
2018-09-02 11:46:57,067 : INFO : min_count=20 retains 613 unique words (45% of original 1337, drops 724)
2018-09-02 11:46:57,069 : INFO : min_count=20 leaves 225765 word corpus (98% of original 229662, drops 3897)
2018-09-02 11:46:57,072 : INFO : deleting the raw counts dictionary of 1337 items
2018-09-02 11:46:57,074 : INFO : sample=0.001 downsamples 78 most-common words
2018-09-02 11:46:57,074 : INFO : downsampling leaves estimated 126697 word corpus (56.1% of prior 225765)
201

In [46]:
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d391,n10,hs,w8,mc20,s0.001,t4)


In [47]:
doc_vectorizer.corpus_count*0.03

91.95

In [48]:
doc_vectorizer.iter

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


5

In [49]:
import time
start = time.time()
for epoch in range(10):
    doc_vectorizer.train(sentences, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay
end = time.time()
print("During Time: {}".format(end-start))

/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.
2018-09-02 11:47:29,274 : INFO : training model with 4 workers on 613 vocabulary and 391 features, using sg=1 hs=1 sample=0.001 negative=10 window=8
2018-09-02 11:47:30,405 : INFO : EPOCH 1 - PROGRESS: at 21.17% examples, 44661 words/s, in_qsize 8, out_qsize 0
2018-09-02 11:47:31,277 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:47:31,462 : INFO : EPOCH 1 - PROGRESS: at 83.16% examples, 54242 words/s, in_qsize 2, out_qsize 1
2018-09-02 11:47:31,463 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:47:31,468 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:47:31,477 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:47:31,478 : I

2018-09-02 11:47:55,073 : INFO : EPOCH 1 - PROGRESS: at 75.69% examples, 48395 words/s, in_qsize 3, out_qsize 1
2018-09-02 11:47:55,075 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:47:55,246 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:47:55,252 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:47:55,311 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:47:55,312 : INFO : EPOCH - 1 : training on 229662 raw words (129655 effective words) took 2.6s, 50500 effective words/s
2018-09-02 11:47:56,688 : INFO : EPOCH 2 - PROGRESS: at 21.92% examples, 36658 words/s, in_qsize 7, out_qsize 0
2018-09-02 11:47:57,648 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:47:57,734 : INFO : EPOCH 2 - PROGRESS: at 81.89% examples, 48959 words/s, in_qsize 2, out_qsize 1
2018-09-02 11:47:57,735 : INFO : worker thread finished; awaiting finish of

2018-09-02 11:48:19,851 : INFO : EPOCH - 1 : training on 229662 raw words (129702 effective words) took 2.4s, 54611 effective words/s
2018-09-02 11:48:20,993 : INFO : EPOCH 2 - PROGRESS: at 21.17% examples, 44204 words/s, in_qsize 7, out_qsize 0
2018-09-02 11:48:21,871 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:48:22,096 : INFO : EPOCH 2 - PROGRESS: at 86.33% examples, 52872 words/s, in_qsize 2, out_qsize 1
2018-09-02 11:48:22,097 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:48:22,098 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:48:22,099 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:48:22,100 : INFO : EPOCH - 2 : training on 229662 raw words (129693 effective words) took 2.2s, 57810 effective words/s
2018-09-02 11:48:23,304 : INFO : EPOCH 3 - PROGRESS: at 21.92% examples, 41840 words/s, in_qsize 8, out_qsize 0
2018-09-02 11:48:24,185 : INFO :

2018-09-02 11:48:45,726 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:48:45,757 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:48:45,823 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:48:45,824 : INFO : EPOCH - 2 : training on 229662 raw words (129751 effective words) took 2.4s, 54178 effective words/s
2018-09-02 11:48:46,942 : INFO : EPOCH 3 - PROGRESS: at 13.15% examples, 24907 words/s, in_qsize 8, out_qsize 0
2018-09-02 11:48:47,943 : INFO : EPOCH 3 - PROGRESS: at 34.88% examples, 34222 words/s, in_qsize 8, out_qsize 0
2018-09-02 11:48:48,545 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:48:48,710 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:48:48,750 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:48:48,756 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-

2018-09-02 11:49:12,491 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:49:12,659 : INFO : EPOCH 3 - PROGRESS: at 81.89% examples, 51860 words/s, in_qsize 2, out_qsize 1
2018-09-02 11:49:12,660 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-09-02 11:49:12,683 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-02 11:49:12,709 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-02 11:49:12,710 : INFO : EPOCH - 3 : training on 229662 raw words (129747 effective words) took 2.3s, 55628 effective words/s
2018-09-02 11:49:13,854 : INFO : EPOCH 4 - PROGRESS: at 21.17% examples, 44134 words/s, in_qsize 7, out_qsize 0
2018-09-02 11:49:14,739 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-09-02 11:49:14,881 : INFO : EPOCH 4 - PROGRESS: at 81.89% examples, 54727 words/s, in_qsize 2, out_qsize 1
2018-09-02 11:49:14,882 : INFO : worker thread finished; awaiting finish of

During Time: 119.23198103904724


In [50]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in sentences]

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train , y , test_size=0.33, random_state=42)

In [52]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.807312252964


In [53]:
from sklearn.neural_network import MLPClassifier

In [54]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=10,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [55]:
start = time.time()
mlp_clf.fit(X_train, y)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.58986401
Iteration 2, loss = 0.39349004
Iteration 3, loss = 0.34249328
Iteration 4, loss = 0.31402776
Iteration 5, loss = 0.29559513
Iteration 6, loss = 0.28058410
Iteration 7, loss = 0.26707563
Iteration 8, loss = 0.25575315
Iteration 9, loss = 0.24410927
Iteration 10, loss = 0.23320133
Time: 0.169047s


/Users/yonggeol/anaconda3/envs/py/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [56]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.916
